In [ ]:
!pip install -q diffusers transformers accelerate streamlit
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.

In [ ]:
%%writefile app.py
import streamlit as st
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import time
import random
import io
import base64
from PIL import Image
import os
from datetime import datetime

# Set page configuration
st.set_page_config(
    page_title="AI Dream Canvas | Stable Diffusion Generator",
    page_icon="🎨",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Apply custom CSS
def load_css():
    st.markdown("""
    <style>
        .main .block-container {
            padding-top: 2rem;
            padding-bottom: 2rem;
        }
        h1, h2, h3 {
            color: #1E88E5;
        }
        .stButton > button {
            background-color: #1E88E5;
            color: white;
            font-weight: bold;
            border-radius: 8px;
            padding: 0.5rem 1rem;
            transition: all 0.3s;
        }
        .stButton > button:hover {
            background-color: #1565C0;
            box-shadow: 0 4px 8px rgba(0,0,0,0.1);
        }
        .image-card {
            border: 1px solid #e0e0e0;
            border-radius: 10px;
            padding: 1rem;
            background-color: white;
            box-shadow: 0 4px 6px rgba(0,0,0,0.05);
        }
        .sidebar .block-container {
            padding-top: 2rem;
        }
        .stSlider > div > div {
            color: #1E88E5 !important;
        }
        .gallery-image {
            border-radius: 8px;
            transition: transform 0.3s;
        }
        .gallery-image:hover {
            transform: scale(1.02);
        }
        .status-indicator {
            padding: 0.3rem 0.6rem;
            border-radius: 20px;
            font-size: 0.8rem;
            font-weight: bold;
        }
        .status-ready {
            background-color: #4CAF50;
            color: white;
        }
        .status-busy {
            background-color: #FFC107;
            color: black;
        }
        footer {visibility: hidden;}
    </style>
    """, unsafe_allow_html=True)

load_css()

# Initialize session state variables
if 'history' not in st.session_state:
    st.session_state.history = []
if 'current_image' not in st.session_state:
    st.session_state.current_image = None
if 'current_prompt' not in st.session_state:
    st.session_state.current_prompt = ""
if 'model_loaded' not in st.session_state:
    st.session_state.model_loaded = False
if 'pipe' not in st.session_state:
    st.session_state.pipe = None
if 'gallery_view' not in st.session_state:
    st.session_state.gallery_view = False
if 'saved_images' not in st.session_state:
    # Create directory to save images if it doesn't exist
    if not os.path.exists("generated_images"):
        os.makedirs("generated_images")
    st.session_state.saved_images = []

# Function to get image download link
def get_image_download_link(img, filename, text):
    buffered = io.BytesIO()
    img.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode()
    href = f'<a href="data:image/png;base64,{img_str}" download="{filename}" style="display: inline-block; padding: 0.5rem 1rem; background-color: #4CAF50; color: white; text-decoration: none; border-radius: 4px; text-align: center; font-weight: bold;">{text}</a>'
    return href

# Function to save image to disk
def save_image(image, prompt):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"generated_images/sd_{timestamp}.png"
    image.save(filename)
    return {
        "filename": filename,
        "prompt": prompt,
        "timestamp": timestamp,
        "image": image
    }

# Function to load model - adapted for Colab
@st.cache_resource
def load_model(model_id, scheduler_type, torch_dtype):
    if scheduler_type == "DPM++ 2M":
        scheduler = DPMSolverMultistepScheduler.from_pretrained(
            model_id, subfolder="scheduler", solver_order=2
        )
        pipe = StableDiffusionPipeline.from_pretrained(
            model_id,
            scheduler=scheduler,
            torch_dtype=torch_dtype
        )
    else:
        pipe = StableDiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=torch_dtype
        )

    # Move to GPU if available
    if torch.cuda.is_available():
        pipe = pipe.to("cuda")

        # Enable attention slicing for lower memory usage
        pipe.enable_attention_slicing()

        # Safely check if we can use memory efficient attention
        # Skip xformers since it can cause issues in Colab

    return pipe

# Generate random prompts for inspiration
def get_random_prompt():
    styles = ["photorealistic", "oil painting", "watercolor", "digital art", "pencil sketch", "3D render", "anime style"]
    subjects = ["mountain landscape", "futuristic cityscape", "magical forest", "ocean sunset", "cosmic nebula", "fantasy character", "cyberpunk street"]
    adjectives = ["vibrant", "mysterious", "ethereal", "dramatic", "peaceful", "dystopian", "surreal"]
    details = ["detailed", "high quality", "intricate", "professional", "8K", "trending on artstation", "award-winning"]

    style = random.choice(styles)
    subject = random.choice(subjects)
    adjective = random.choice(adjectives)
    detail = random.choice(details)

    return f"A {adjective} {subject}, {style}, {detail}"

# Function to generate image
def generate_image(prompt, negative_prompt, num_inference_steps, guidance_scale, width, height, seed, model_id, scheduler_type):
    try:
        # Select device and data type
        torch_device = "cuda" if torch.cuda.is_available() else "cpu"
        torch_dtype = torch.float16 if torch_device == "cuda" else torch.float32

        # Load or get model
        if not st.session_state.model_loaded or st.session_state.current_model_id != model_id or st.session_state.current_scheduler != scheduler_type:
            with st.spinner("Loading model... This may take a minute."):
                st.session_state.pipe = load_model(model_id, scheduler_type, torch_dtype)
                st.session_state.model_loaded = True
                st.session_state.current_model_id = model_id
                st.session_state.current_scheduler = scheduler_type

        pipe = st.session_state.pipe

        # Set up generator for reproducibility
        generator = None
        if seed != 0:
            generator = torch.Generator(device=torch_device).manual_seed(seed)

        # Generate the image
        start_time = time.time()
        output = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            width=width,
            height=height,
            generator=generator
        )
        end_time = time.time()

        # Get the image
        image = output.images[0]

        # Calculate generation time
        generation_time = end_time - start_time

        # Add to history
        image_info = {
            "prompt": prompt,
            "negative_prompt": negative_prompt,
            "inference_steps": num_inference_steps,
            "guidance_scale": guidance_scale,
            "dimensions": f"{width}x{height}",
            "seed": seed if seed != 0 else "random",
            "model": model_id.split("/")[-1],
            "generation_time": f"{generation_time:.2f}s",
            "image": image,
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

        st.session_state.history.append(image_info)
        st.session_state.current_image = image
        st.session_state.current_prompt = prompt

        return image, image_info

    except Exception as e:
        st.error(f"Error generating image: {str(e)}")
        return None, None

# App header
def app_header():
    col1, col2 = st.columns([6, 1])
    with col1:
        st.markdown("# 🎨 AI Dream Canvas")
        st.markdown("#### Transform your imagination into stunning visuals with Stable Diffusion")

    with col2:
        device = "GPU" if torch.cuda.is_available() else "CPU"
        status_class = "status-ready" if device == "GPU" else "status-busy"
        st.markdown(f"""
        <div style="text-align: right; margin-top: 1rem;">
            <span class="status-indicator {status_class}">
                {device}
            </span>
        </div>
        """, unsafe_allow_html=True)

# Main app
def main():
    app_header()

    # Sidebar - Model Settings
    with st.sidebar:
        st.markdown("## 🔧 Model Settings")

        # Model selection
        model_options = {
            "CompVis/stable-diffusion-v1-4": "Stable Diffusion v1.4",
            "runwayml/stable-diffusion-v1-5": "Stable Diffusion v1.5"
        }
        model_id = st.selectbox(
            "Model",
            options=list(model_options.keys()),
            format_func=lambda x: model_options[x],
            index=1  # Default to v1.5
        )

        # Scheduler selection
        scheduler_type = st.selectbox(
            "Scheduler",
            options=["Default", "DPM++ 2M"],
            index=1  # Default to DPM++ 2M
        )

        # Resolution settings
        st.markdown("### 📐 Resolution")
        col1, col2 = st.columns(2)
        with col1:
            width = st.select_slider(
                "Width",
                options=[384, 512, 576, 640],
                value=512
            )
        with col2:
            height = st.select_slider(
                "Height",
                options=[384, 512, 576, 640],
                value=512
            )

        st.markdown("### ⚙️ Generation Parameters")

        # Quality vs. Speed slider
        quality_speed = st.slider(
            "Quality vs. Speed",
            min_value=1,
            max_value=5,
            value=3,
            help="1: Fastest (low quality), 5: Highest quality (slow)"
        )

        # Map quality_speed to inference steps
        quality_to_steps = {1: 20, 2: 25, 3: 30, 4: 40, 5: 50}
        num_inference_steps = quality_to_steps[quality_speed]

        # Advanced settings in expander
        with st.expander("Advanced Settings"):
            # Allow direct adjustment of steps
            num_inference_steps = st.slider(
                "Inference Steps",
                min_value=10,
                max_value=75,
                value=num_inference_steps,
                help="Higher values = better quality but slower generation"
            )

            guidance_scale = st.slider(
                "Guidance Scale",
                min_value=1.0,
                max_value=20.0,
                value=7.5,
                step=0.5,
                help="How strongly the image follows the prompt (higher = more faithful)"
            )

            seed = st.number_input(
                "Random Seed",
                value=0,
                help="0 for random, any other number for reproducible results"
            )

        # View Toggle
        st.markdown("### 🖼️ View Options")
        view_toggle = st.radio(
            "Display Mode",
            options=["Generator", "Gallery"],
            horizontal=True,
            index=0 if not st.session_state.gallery_view else 1
        )
        st.session_state.gallery_view = (view_toggle == "Gallery")

        # System info
        device = "CUDA (GPU)" if torch.cuda.is_available() else "CPU"
        memory_info = ""
        if torch.cuda.is_available():
            try:
                gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
                memory_info = f", {gpu_memory:.1f} GB VRAM"
            except:
                memory_info = ""

        st.info(f"Running on: {device}{memory_info}")

        # Colab specific tips
        st.markdown("---")
        st.markdown("#### 💡 Colab Tips")
        st.markdown(
            "- First generation may take longer while model downloads\n"
            "- For stable performance, use Colab with GPU runtime\n"
            "- Larger resolutions require more VRAM"
        )

    # Main content area
    if not st.session_state.gallery_view:
        # Generator View
        col1, col2 = st.columns([5, 6])

        with col1:
            st.markdown("### 🖋 Enter your prompt")

            # Prompt input area with examples
            prompt = st.text_area(
                "Describe the image you want to generate",
                st.session_state.current_prompt or "A futuristic cityscape at sunset with flying cars, detailed digital art",
                height=100
            )

            # Random prompt button
            col1a, col1b = st.columns([1, 1])
            with col1a:
                if st.button("🎲 Random Prompt", use_container_width=True):
                    prompt = get_random_prompt()
                    st.session_state.current_prompt = prompt
                    st.rerun()

            # Example prompts
            with st.expander("✨ Prompt Examples"):
                example_prompts = [
                    "A serene Japanese garden with cherry blossoms, 4K, highly detailed",
                    "Cyberpunk portrait of a young woman with neon lights, digital art",
                    "Mystical forest with glowing mushrooms and fairy lights, fantasy concept art",
                    "An astronaut riding a horse on Mars, photorealistic style"
                ]

                for i, ex_prompt in enumerate(example_prompts):
                    if st.button(f"Example {i+1}", key=f"example_{i}", use_container_width=True):
                        prompt = ex_prompt
                        st.session_state.current_prompt = prompt
                        st.rerun()

            # Negative prompt
            negative_prompt = st.text_area(
                "Negative prompt (things to avoid)",
                "blurry, bad anatomy, bad hands, cropped, worst quality, low quality",
                height=80
            )

            # Generation button
            generate_button = st.button(
                "🚀 Generate Image",
                type="primary",
                use_container_width=True
            )

        with col2:
            # Display current image or placeholder
            st.markdown("### 🖼️ Generated Image")

            image_container = st.container()
            with image_container:
                if st.session_state.current_image is not None:
                    st.image(
                        st.session_state.current_image,
                        caption=st.session_state.current_prompt,
                        use_column_width=True
                    )

                    # Image info and download buttons
                    col_save, col_download = st.columns(2)
                    with col_save:
                        if st.button("💾 Save to Gallery", use_container_width=True):
                            saved_info = save_image(st.session_state.current_image, st.session_state.current_prompt)
                            st.session_state.saved_images.append(saved_info)
                            st.success("Image saved to gallery!")

                    with col_download:
                        download_link = get_image_download_link(
                            st.session_state.current_image,
                            "stable_diffusion_image.png",
                            "📥 Download Image"
                        )
                        st.markdown(download_link, unsafe_allow_html=True)

                    # Show generation info if available
                    if st.session_state.history:
                        latest = st.session_state.history[-1]
                        with st.expander("Generation Details", expanded=False):
                            info_col1, info_col2 = st.columns(2)
                            with info_col1:
                                st.markdown(f"**Model:** {latest['model']}")
                                st.markdown(f"**Dimensions:** {latest['dimensions']}")
                                st.markdown(f"**Seed:** {latest['seed']}")
                            with info_col2:
                                st.markdown(f"**Steps:** {latest['inference_steps']}")
                                st.markdown(f"**Guidance Scale:** {latest['guidance_scale']}")
                                st.markdown(f"**Generation Time:** {latest['generation_time']}")
                else:
                    # Placeholder for when no image is generated yet
                    st.markdown(
                        """
                        <div style="
                            border: 2px dashed #BDBDBD;
                            border-radius: 10px;
                            padding: 7rem 1rem;
                            text-align: center;
                            color: #9E9E9E;
                        ">
                            <h3>Your creation will appear here</h3>
                            <p>Enter a prompt and click "Generate Image" to begin</p>
                        </div>
                        """,
                        unsafe_allow_html=True
                    )

        # Generation history
        if st.session_state.history:
            st.markdown("---")
            st.markdown("### 📜 Recent Generations")

            # Display the last 5 generations in a horizontal scroll
            history_items = list(reversed(st.session_state.history))[:5]

            cols = st.columns(min(len(history_items), 5))
            for i, history_item in enumerate(history_items[:5]):
                with cols[i]:
                    st.image(
                        history_item["image"],
                        use_column_width=True,
                        caption=history_item["prompt"][:40] + "..." if len(history_item["prompt"]) > 40 else history_item["prompt"],
                        output_format="PNG",
                        clamp=True
                    )

        # Handle image generation when button is clicked
        if generate_button:
            with st.spinner("🔮 Bringing your imagination to life..."):
                image, _ = generate_image(
                    prompt=prompt,
                    negative_prompt=negative_prompt,
                    num_inference_steps=num_inference_steps,
                    guidance_scale=guidance_scale,
                    width=width,
                    height=height,
                    seed=seed,
                    model_id=model_id,
                    scheduler_type=scheduler_type
                )
                if image:
                    st.rerun()

    else:
        # Gallery View
        st.markdown("### 🖼️ Image Gallery")

        if not st.session_state.saved_images:
            st.info("Your gallery is empty. Generate and save images to see them here.")
        else:
            # Display saved images in a grid
            num_cols = 3
            saved_images = list(reversed(st.session_state.saved_images))

            for i in range(0, len(saved_images), num_cols):
                cols = st.columns(num_cols)
                for j in range(num_cols):
                    if i + j < len(saved_images):
                        img_info = saved_images[i + j]
                        with cols[j]:
                            st.image(
                                img_info["image"],
                                caption=img_info["prompt"][:40] + "..." if len(img_info["prompt"]) > 40 else img_info["prompt"],
                                use_column_width=True,
                                output_format="PNG",
                                clamp=True
                            )

                            # Show when the image was created
                            st.caption(f"Created: {img_info['timestamp']}")

                            # Download button for each gallery image
                            download_link = get_image_download_link(
                                img_info["image"],
                                f"sd_image_{img_info['timestamp']}.png",
                                "📥 Download"
                            )
                            st.markdown(download_link, unsafe_allow_html=True)

if __name__ == "__main__":
    main()

Writing app.py


In [ ]:
# Start the Streamlit app
!nohup streamlit run app.py --server.port 8501 &

# Start Cloudflare tunnel
!./cloudflared tunnel --url http://localhost:8501 --no-autoupdate


nohup: appending output to 'nohup.out'
nohup: failed to run command 'streamlit': No such file or directory
/bin/bash: line 1: ./cloudflared: No such file or directory


In [ ]:
# Install required packages
!pip install flask tensorflow keras numpy pandas scikit-learn

# Import necessary libraries
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Embedding
import json
import os
from flask import Flask, request, jsonify, render_template_string

# Define models
class FinancialAlert:
    def __init__(self, user_id, title, amount, due_date, frequency='MONTHLY'):
        self.user_id = user_id
        self.title = title
        self.amount = amount
        self.due_date = datetime.strptime(due_date, '%Y-%m-%d').date()
        self.frequency = frequency
        self.is_active = True

class Expense:
    def __init__(self, user_id, amount, category, date):
        self.user_id = user_id
        self.amount = amount
        self.category = category
        self.date = datetime.strptime(date, '%Y-%m-%d').date()

# Mock database
class Database:
    def __init__(self):
        self.alerts = []
        self.expenses = []
        self.users = {}

    def add_alert(self, alert):
        self.alerts.append(alert)
        return alert

    def get_user_alerts(self, user_id):
        return [alert for alert in self.alerts if alert.user_id == user_id and alert.is_active]

    def add_expense(self, expense):
        self.expenses.append(expense)
        return expense

    def get_user_expenses(self, user_id, category=None):
        if category:
            return [exp for exp in self.expenses if exp.user_id == user_id and exp.category == category]
        return [exp for exp in self.expenses if exp.user_id == user_id]

# Alert Service
class AlertService:
    def __init__(self, db):
        self.db = db

    def predict_payment_date(self, user_id, alert_type):
        expenses = self.db.get_user_expenses(user_id, alert_type)
        if len(expenses) < 3:
            return None

        dates = np.array([(exp.date - expenses[0].date).days for exp in expenses]).reshape(-1, 1)
        amounts = np.array([exp.amount for exp in expenses])

        model = LinearRegression()
        model.fit(dates, amounts)

        next_date = dates[-1] + 30
        predicted_amount = model.predict([[next_date[0]]])[0]

        return {
            'predicted_date': expenses[0].date + timedelta(days=int(next_date[0])),
            'predicted_amount': round(predicted_amount, 2)
        }

    def get_due_alerts(self, user_id):
        today = datetime.now().date()
        return [alert for alert in self.db.get_user_alerts(user_id)
                if 0 <= (alert.due_date - today).days <= 7]

# Chatbot
class FinancialChatbot:
    def __init__(self):
        self.db = Database()
        self.alert_service = AlertService(self.db)

    def handle_alert_command(self, statement, user_id):
        if statement.lower() == "show my alerts":
            alerts = self.alert_service.get_due_alerts(user_id)
            if not alerts:
                return "You have no upcoming payments due."

            response = "Here are your upcoming payments:\n"
            for alert in alerts:
                days_until_due = (alert.due_date - datetime.now().date()).days
                response += f"- {alert.title}: ${alert.amount} due in {days_until_due} days ({alert.due_date})\n"
            return response

        if statement.lower().startswith("create alert:"):
            try:
                _, details = statement.split(":", 1)
                title, amount, due_date, frequency = [x.strip() for x in details.split("|")]

                alert = FinancialAlert(
                    user_id=user_id,
                    title=title,
                    amount=float(amount),
                    due_date=due_date,
                    frequency=frequency.upper()
                )
                self.db.add_alert(alert)
                return f"Alert created successfully for {title}"
            except Exception as e:
                return "Invalid alert format. Please use: create alert: title | amount | due_date | frequency"

        return None

    def process_message(self, message, user_id):
        alert_response = self.handle_alert_command(message, user_id)
        if alert_response:
            return alert_response
        return "I'm a financial chatbot. I can help you with alerts and expenses."

# Initialize Flask app
app = Flask(__name__)
chatbot = FinancialChatbot()

# HTML template
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Financial Chatbot</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            background-color: #f5f5f5;
        }
        .chat-container {
            max-width: 800px;
            margin: 0 auto;
            background: white;
            border-radius: 10px;
            box-shadow: 0 0 10px rgba(0,0,0,0.1);
            padding: 20px;
        }
        .chat-header {
            text-align: center;
            padding: 10px;
            background: #2c3e50;
            color: white;
            border-radius: 5px;
            margin-bottom: 20px;
        }
        .chat-messages {
            height: 400px;
            overflow-y: auto;
            padding: 10px;
            border: 1px solid #ddd;
            border-radius: 5px;
            margin-bottom: 20px;
        }
        .message {
            margin: 10px 0;
            padding: 10px;
            border-radius: 5px;
        }
        .user-message {
            background: #e3f2fd;
            margin-left: 20%;
        }
        .bot-message {
            background: #f1f1f1;
            margin-right: 20%;
        }
        .input-container {
            display: flex;
            gap: 10px;
        }
        input[type="text"] {
            flex: 1;
            padding: 10px;
            border: 1px solid #ddd;
            border-radius: 5px;
        }
        button {
            padding: 10px 20px;
            background: #2c3e50;
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }
        button:hover {
            background: #34495e;
        }
    </style>
</head>
<body>
    <div class="chat-container">
        <div class="chat-header">
            <h2>Financial Chatbot</h2>
        </div>
        <div class="chat-messages" id="chat-messages">
            <div class="message bot-message">
                Hello! I'm your financial assistant. How can I help you today?
            </div>
        </div>
        <div class="input-container">
            <input type="text" id="user-input" placeholder="Type your message here...">
            <button onclick="sendMessage()">Send</button>
        </div>
    </div>

    <script>
        function sendMessage() {
            const input = document.getElementById('user-input');
            const message = input.value.trim();
            if (message) {
                // Add user message to chat
                addMessage(message, 'user');
                input.value = '';

                // Send message to backend
                fetch('/chat', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify({ message: message, user_id: 1 })
                })
                .then(response => response.json())
                .then(data => {
                    addMessage(data.response, 'bot');
                });
            }
        }

        function addMessage(text, sender) {
            const chatMessages = document.getElementById('chat-messages');
            const messageDiv = document.createElement('div');
            messageDiv.className = `message ${sender}-message`;
            messageDiv.textContent = text;
            chatMessages.appendChild(messageDiv);
            chatMessages.scrollTop = chatMessages.scrollHeight;
        }

        // Handle Enter key
        document.getElementById('user-input').addEventListener('keypress', function(e) {
            if (e.key === 'Enter') {
                sendMessage();
            }
        });
    </script>
</body>
</html>
"""

@app.route('/')
def home():
    return render_template_string(HTML_TEMPLATE)

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json
    message = data['message']
    user_id = data['user_id']
    response = chatbot.process_message(message, user_id)
    return jsonify({'response': response})

# Run the Flask app
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
var='''Hi'''
print(var)

Hi


In [ ]:
'''Hi'''
print("Heloo")


Heloo


In [ ]:
a=10
b=10
print(id(a))
print(id(b))
a=20
print(a)
print(id(a))

10751144
10751144
20
10751464


In [ ]:
a=-6
b=-6
print(id(a))
print(id(b))

137991516628048
137991516629680
